# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for a battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset from Ironhack's database
pokemon = pd.read_csv("../Datasets as CSV/Pokemon.csv")
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
"""Since the HP, Attack, Defense, Sp. Atk, Sp. Def, and Speed 
are all attack and none of them seems to be a negative value, 
it will make sense that it represents the sum of all the columns"""

#If we want to prove that our hypothesis is correct : 
hypothesis = pd.DataFrame({"Name": pokemon.Name, "Total": pokemon.Total , "Hypothesis" : (pokemon.HP + pokemon.Attack + pokemon.Defense + pokemon["Sp. Atk"] + pokemon["Sp. Def"] + pokemon.Speed)})
list_hypothesis = hypothesis.Hypothesis.tolist()

#Now we want to check wether there are rows in Total that are not equal in Hypothesis
pokemon.loc[~pokemon["Total"].isin(list_hypothesis)]
#byusing ~something.isin we are asking for all the rows in pokemon Total that are not in Hypothesis. If we do the reverse, the total rows and the rows in total should be equalç
pokemon.loc[pokemon["Total"].isin(list_hypothesis)]
#As expected, the rows where total equal to the values in the list we made with the sum, is 800, which is the total number of rows.
                           

#                   Name   Type 1  Type 2  Total  HP  Attack  Defense  \
0      1              Bulbasaur    Grass  Poison    318  45      49       49   
1      2                Ivysaur    Grass  Poison    405  60      62       63   
2      3               Venusaur    Grass  Poison    525  80      82       83   
3      3  VenusaurMega Venusaur    Grass  Poison    625  80     100      123   
4      4             Charmander     Fire     NaN    309  39      52       43   
..   ...                    ...      ...     ...    ...  ..     ...      ...   
795  719                Diancie     Rock   Fairy    600  50     100      150   
796  719    DiancieMega Diancie     Rock   Fairy    700  50     160      110   
797  720    HoopaHoopa Confined  Psychic   Ghost    600  80     110       60   
798  720     HoopaHoopa Unbound  Psychic    Dark    680  80     160       60   
799  721              Volcanion     Fire   Water    600  80     110      120   

     Sp. Atk  Sp. Def  Speed  Generation  Legendary  
0         65       65     45           1      False  
1         80       80     60           1      False  
2        100      100     80           1      False  
3        122      120     80           1      False  
4         60       50     65           1      False  
..       ...      ...    ...         ...        ...  
795      100      150     50           6       True  
796      160      110    110           6       True  
797      150      130     70           6       True  
798      170      130     80           6       True  
799      130       90     70           6       True  

[800 rows x 13 columns]

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [4]:
type_numerical = pd.get_dummies(pokemon["Type 1"])

type_numerical.head()

Bug  Dark  Dragon  Electric  Fairy  Fighting  Fire  Flying  Ghost  Grass  \
0    0     0       0         0      0         0     0       0      0      1   
1    0     0       0         0      0         0     0       0      0      1   
2    0     0       0         0      0         0     0       0      0      1   
3    0     0       0         0      0         0     0       0      0      1   
4    0     0       0         0      0         0     1       0      0      0   

   Ground  Ice  Normal  Poison  Psychic  Rock  Steel  Water  
0       0    0       0       0        0     0      0      0  
1       0    0       0       0        0     0      0      0  
2       0    0       0       0        0     0      0      0  
3       0    0       0       0        0     0      0      0  
4       0    0       0       0        0     0      0      0

In [5]:
pokemon = pd.concat([pokemon,type_numerical], axis=1)
pokemon.head()

#                   Name Type 1  Type 2  Total  HP  Attack  Defense  \
0  1              Bulbasaur  Grass  Poison    318  45      49       49   
1  2                Ivysaur  Grass  Poison    405  60      62       63   
2  3               Venusaur  Grass  Poison    525  80      82       83   
3  3  VenusaurMega Venusaur  Grass  Poison    625  80     100      123   
4  4             Charmander   Fire     NaN    309  39      52       43   

   Sp. Atk  Sp. Def  ...  Ghost  Grass  Ground  Ice  Normal  Poison  Psychic  \
0       65       65  ...      0      1       0    0       0       0        0   
1       80       80  ...      0      1       0    0       0       0        0   
2      100      100  ...      0      1       0    0       0       0        0   
3      122      120  ...      0      1       0    0       0       0        0   
4       60       50  ...      0      0       0    0       0       0        0   

   Rock  Steel  Water  
0     0      0      0  
1     0      0      0  
2     0      0      0  
3     0      0      0  
4     0      0      0  

[5 rows x 31 columns]

In [6]:
pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary', 'Bug', 'Dark',
       'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost',
       'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock',
       'Steel', 'Water'],
      dtype='object')

In [7]:
pokemon = pokemon[['#', 'Name', 'Type 1', 'Type 2','HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary','Total', 'Bug', 'Dark',
       'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost',
       'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock',
       'Steel', 'Water']]
pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk',
       'Sp. Def', 'Speed', 'Generation', 'Legendary', 'Total', 'Bug', 'Dark',
       'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost',
       'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock',
       'Steel', 'Water'],
      dtype='object')

## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [10]:
pokemon.iloc[:,[12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]].corr()

#Total has the most positive correlations with : 
#Fire
#Fighting
#Flying 
#Dark

Total       Bug      Dark    Dragon  Electric     Fairy  \
Total     1.000000 -0.143957  0.017818  0.196532  0.016715 -0.026948   
Bug      -0.143957  1.000000 -0.061686 -0.062713 -0.074119 -0.045270   
Dark      0.017818 -0.061686  1.000000 -0.040984 -0.048438 -0.029584   
Dragon    0.196532 -0.062713 -0.040984  1.000000 -0.049245 -0.030077   
Electric  0.016715 -0.074119 -0.048438 -0.049245  1.000000 -0.035547   
Fairy    -0.026948 -0.045270 -0.029584 -0.030077 -0.035547  1.000000   
Fighting -0.029086 -0.057419 -0.037524 -0.038149 -0.045088 -0.027538   
Fire      0.050527 -0.081006 -0.052938 -0.053820 -0.063609 -0.038850   
Flying    0.029504 -0.021779 -0.014233 -0.014470 -0.017102 -0.010445   
Ghost     0.007594 -0.062713 -0.040984 -0.041667 -0.049245 -0.030077   
Grass    -0.036057 -0.095138 -0.062173 -0.063209 -0.074706 -0.045628   
Ground    0.004082 -0.062713 -0.040984 -0.041667 -0.049245 -0.030077   
Ice      -0.002412 -0.054031 -0.035310 -0.035898 -0.042427 -0.025913   
Normal   -0.104150 -0.114792 -0.075017 -0.076267 -0.090138 -0.055054   
Poison   -0.057123 -0.058511 -0.038237 -0.038874 -0.045945 -0.028062   
Psychic   0.094364 -0.085096 -0.055611 -0.056538 -0.066820 -0.040812   
Rock      0.037524 -0.074119 -0.048438 -0.049245 -0.058201 -0.035547   
Steel     0.082000 -0.057419 -0.037524 -0.038149 -0.045088 -0.027538   
Water    -0.015640 -0.123960 -0.081009 -0.082359 -0.097338 -0.059451   

          Fighting      Fire    Flying     Ghost     Grass    Ground  \
Total    -0.029086  0.050527  0.029504  0.007594 -0.036057  0.004082   
Bug      -0.057419 -0.081006 -0.021779 -0.062713 -0.095138 -0.062713   
Dark     -0.037524 -0.052938 -0.014233 -0.040984 -0.062173 -0.040984   
Dragon   -0.038149 -0.053820 -0.014470 -0.041667 -0.063209 -0.041667   
Electric -0.045088 -0.063609 -0.017102 -0.049245 -0.074706 -0.049245   
Fairy    -0.027538 -0.038850 -0.010445 -0.030077 -0.045628 -0.030077   
Fighting  1.000000 -0.049277 -0.013248 -0.038149 -0.057873 -0.038149   
Fire     -0.049277  1.000000 -0.018691 -0.053820 -0.081647 -0.053820   
Flying   -0.013248 -0.018691  1.000000 -0.014470 -0.021951 -0.014470   
Ghost    -0.038149 -0.053820 -0.014470  1.000000 -0.063209 -0.041667   
Grass    -0.057873 -0.081647 -0.021951 -0.063209  1.000000 -0.063209   
Ground   -0.038149 -0.053820 -0.014470 -0.041667 -0.063209  1.000000   
Ice      -0.032868 -0.046369 -0.012467 -0.035898 -0.054458 -0.035898   
Normal   -0.069829 -0.098513 -0.026486 -0.076267 -0.115700 -0.076267   
Poison   -0.035593 -0.050214 -0.013500 -0.038874 -0.058974 -0.038874   
Psychic  -0.051765 -0.073029 -0.019634 -0.056538 -0.085769 -0.056538   
Rock     -0.045088 -0.063609 -0.017102 -0.049245 -0.074706 -0.049245   
Steel    -0.034929 -0.049277 -0.013248 -0.038149 -0.057873 -0.038149   
Water    -0.075406 -0.106381 -0.028601 -0.082359 -0.124940 -0.082359   

               Ice    Normal    Poison   Psychic      Rock     Steel     Water  
Total    -0.002412 -0.104150 -0.057123  0.094364  0.037524  0.082000 -0.015640  
Bug      -0.054031 -0.114792 -0.058511 -0.085096 -0.074119 -0.057419 -0.123960  
Dark     -0.035310 -0.075017 -0.038237 -0.055611 -0.048438 -0.037524 -0.081009  
Dragon   -0.035898 -0.076267 -0.038874 -0.056538 -0.049245 -0.038149 -0.082359  
Electric -0.042427 -0.090138 -0.045945 -0.066820 -0.058201 -0.045088 -0.097338  
Fairy    -0.025913 -0.055054 -0.028062 -0.040812 -0.035547 -0.027538 -0.059451  
Fighting -0.032868 -0.069829 -0.035593 -0.051765 -0.045088 -0.034929 -0.075406  
Fire     -0.046369 -0.098513 -0.050214 -0.073029 -0.063609 -0.049277 -0.106381  
Flying   -0.012467 -0.026486 -0.013500 -0.019634 -0.017102 -0.013248 -0.028601  
Ghost    -0.035898 -0.076267 -0.038874 -0.056538 -0.049245 -0.038149 -0.082359  
Grass    -0.054458 -0.115700 -0.058974 -0.085769 -0.074706 -0.057873 -0.124940  
Ground   -0.035898 -0.076267 -0.038874 -0.056538 -0.049245 -0.038149 -0.082359  
Ice       1.000000 -0.065708 -0.033492 -0.048710 -0.042427 -0.032868 -0.070956

In [ ]:
#The best two pokemons are 
# Fire pokemons fighting
# Dark flying pokemons

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?